In [1]:
import numpy as np
import pandas as pd
test = pd.read_csv(
    'data/ann-test.data', 
    sep="\s+",
    header=None,
    # names=columns
)

train = pd.read_csv(
    'data/ann-train.data', 
    sep="\s+",
    header=None,
    # names=columns
)
X = pd.concat([train, test])
y = X.iloc[:, -1]
X = X.drop(21, axis=1)
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,0.73,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0.00060,0.015,0.120,0.082,0.146
1,0.24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00025,0.030,0.143,0.133,0.108
2,0.47,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00190,0.024,0.102,0.131,0.078
3,0.64,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00090,0.017,0.077,0.090,0.085
4,0.23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00025,0.026,0.139,0.090,0.153


In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import label_binarize

# Y = label_binarize(y, classes=[1, 2, 3])

scores = cross_validate(
    LogisticRegression(solver="lbfgs"), 
    X[y < 3], 
    y[y < 3], 
    cv=5, 
    return_train_score=False,
    scoring=("roc_auc", "recall", "average_precision", "accuracy")
)

pd.DataFrame(scores).sort_index(axis=1)

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
3 
Y = label_binarize(y, classes=[1, 2, 3])
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=0)

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score

estimators = [
    LogisticRegression(solver="lbfgs", C=0.5, multi_class="auto", random_state=0),
    LogisticRegression(solver="lbfgs", C=5, multi_class="auto", random_state=0),
    LogisticRegression(solver="lbfgs", C=50, multi_class="auto", random_state=0),
    LogisticRegression(solver="lbfgs", C=0.5, multi_class="auto", class_weight="balanced", random_state=0),
    LogisticRegression(solver="lbfgs", C=5, multi_class="auto", class_weight="balanced", random_state=0),
    LogisticRegression(solver="lbfgs", C=50, multi_class="auto", class_weight="balanced", random_state=0),
]

metrics = []
for estimator in estimators:
    classifier = OneVsRestClassifier(
        estimator,
        n_jobs=-1
    )
    classifier.fit(X_train, y_train)
    y_score = classifier.decision_function(X_test)
    metrics.append({
        'accuracy_score': accuracy_score(y_test, classifier.predict(X_test)),
        'roc_auc': roc_auc_score(y_test, y_score, average="micro"),
        'average_precision': average_precision_score(y_test, y_score, average="micro")
    })
pd.DataFrame(
    metrics, 
    index=["logres", "logres_c5", "logres_c50", "log_ress_balanced", "log_ress_balanced_c5", "log_ress_balanced_c50"]
)

,accuracy_score,average_precision,roc_auc
logres,0.923264,0.954655,0.974209
logres_c5,0.929514,0.970195,0.983820
logres_c50,0.937153,0.978801,0.986753
log_ress_balanced,0.565972,0.757200,0.866745
log_ress_balanced_c5,0.662153,0.813256,0.911773
log_ress_balanced_c50,0.811458,0.887256,0.962913


In [ ]:
from sklearn import metrics
metrics.SCORERS.keys()

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

classifiers = [
    RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0),
    RandomForestClassifier(n_estimators=100, max_depth=2, class_weight="balanced",random_state=0)
]

metrics = []
for classifier in classifiers:
    classifier.fit(X, y)
    y_pred = classifier.predict(X_test)
    metrics.append({
        'accuracy': accuracy_score(y_test, y_pred)
    })
pd.DataFrame(metrics)

,accuracy
0,0.938542
1,0.967361


Changing class_weight is helpful to improve models' metric in both Logistic Regression and Random Classifier.